In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Audio 
from collections import defaultdict
import librosa
from pyts.classification import KNeighborsClassifier
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
df=pd.read_csv("data_dec_scld.csv")
df_y=pd.read_csv("data_dec_angryvocal.csv")
df=pd.concat([df_y.iloc[:,2:3],df.iloc[:,1:] ],axis=1)
df.shape


(2452, 3001)

In [3]:
"""df=df.sample(n=1600, random_state=42)"""

'df=df.sample(n=1600, random_state=42)'

In [4]:
df

,2,0,1,2,3,4,5,6,7,8,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,speech,-0.024939,0.011531,0.003824,-0.001133,0.005707,-0.001231,0.004508,0.002861,-0.006809,...,-0.068274,-0.079312,-0.065353,-0.061078,-0.079842,-0.074183,-0.063330,-0.075893,-0.069719,-0.060970
1,speech,0.064653,0.190734,0.011558,-0.162009,-0.037188,0.224411,0.352410,0.234780,0.036427,...,-0.384608,-0.157868,0.050027,0.112627,0.181900,0.165061,0.070171,0.119354,0.241511,0.217111
2,speech,0.020263,0.014409,0.031487,-0.042241,0.006367,0.035380,-0.031561,0.011531,-0.002462,...,-0.030558,0.052983,0.054366,0.044576,-0.022224,-0.057508,-0.037802,-0.044830,0.002466,0.037955
3,song,-0.019103,0.005012,-0.008641,-0.021548,-0.019538,-0.017426,-0.016571,-0.017745,0.007278,...,-0.036528,-0.026591,0.013671,0.009325,-0.008404,-0.006055,-0.024189,-0.025364,-0.020363,-0.028207
4,speech,0.012969,-0.007232,0.003366,-0.008005,0.003822,0.003396,0.001112,-0.000768,-0.002818,...,0.015396,0.015107,0.012921,0.012818,0.011305,0.008308,0.009650,0.012776,0.014175,0.013939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,song,-0.013229,0.004751,-0.008633,0.000095,-0.011264,-0.005826,0.004365,0.000169,0.001596,...,-0.017778,0.025036,0.007569,-0.003107,0.033189,-0.006684,0.027622,0.025792,0.006912,0.014995
2448,song,0.020862,-0.005326,0.001397,-0.007184,-0.005863,0.009098,0.004302,-0.000174,-0.004781,...,0.001969,0.002012,0.004462,0.000616,-0.002328,0.006944,0.001909,-0.009322,0.005042,-0.003479
2449,song,-0.011058,0.005325,0.005335,-0.004699,-0.003718,-0.001578,0.002487,0.004716,-0.001243,...,0.005099,0.003683,0.006438,-0.000840,-0.010548,-0.010964,-0.005570,0.011386,0.017458,0.006248
2450,song,0.008059,0.001253,0.000403,-0.000832,0.000489,-0.000528,-0.003968,0.000497,0.002776,...,-0.009813,-0.013588,-0.009818,-0.009576,-0.012493,-0.016903,-0.008848,-0.003355,-0.004436,-0.001393


In [5]:
index=df.index

In [6]:
index

RangeIndex(start=0, stop=2452, step=1)

In [7]:
X=df.iloc[:,1:]
y=df.iloc[:,0:1]

In [8]:
X

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,-0.024939,0.011531,0.003824,-0.001133,0.005707,-0.001231,0.004508,0.002861,-0.006809,-0.002455,...,-0.068274,-0.079312,-0.065353,-0.061078,-0.079842,-0.074183,-0.063330,-0.075893,-0.069719,-0.060970
1,0.064653,0.190734,0.011558,-0.162009,-0.037188,0.224411,0.352410,0.234780,0.036427,-0.071036,...,-0.384608,-0.157868,0.050027,0.112627,0.181900,0.165061,0.070171,0.119354,0.241511,0.217111
2,0.020263,0.014409,0.031487,-0.042241,0.006367,0.035380,-0.031561,0.011531,-0.002462,-0.004915,...,-0.030558,0.052983,0.054366,0.044576,-0.022224,-0.057508,-0.037802,-0.044830,0.002466,0.037955
3,-0.019103,0.005012,-0.008641,-0.021548,-0.019538,-0.017426,-0.016571,-0.017745,0.007278,0.013846,...,-0.036528,-0.026591,0.013671,0.009325,-0.008404,-0.006055,-0.024189,-0.025364,-0.020363,-0.028207
4,0.012969,-0.007232,0.003366,-0.008005,0.003822,0.003396,0.001112,-0.000768,-0.002818,-0.004335,...,0.015396,0.015107,0.012921,0.012818,0.011305,0.008308,0.009650,0.012776,0.014175,0.013939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,-0.013229,0.004751,-0.008633,0.000095,-0.011264,-0.005826,0.004365,0.000169,0.001596,-0.004729,...,-0.017778,0.025036,0.007569,-0.003107,0.033189,-0.006684,0.027622,0.025792,0.006912,0.014995
2448,0.020862,-0.005326,0.001397,-0.007184,-0.005863,0.009098,0.004302,-0.000174,-0.004781,0.001159,...,0.001969,0.002012,0.004462,0.000616,-0.002328,0.006944,0.001909,-0.009322,0.005042,-0.003479
2449,-0.011058,0.005325,0.005335,-0.004699,-0.003718,-0.001578,0.002487,0.004716,-0.001243,-0.001279,...,0.005099,0.003683,0.006438,-0.000840,-0.010548,-0.010964,-0.005570,0.011386,0.017458,0.006248
2450,0.008059,0.001253,0.000403,-0.000832,0.000489,-0.000528,-0.003968,0.000497,0.002776,0.000819,...,-0.009813,-0.013588,-0.009818,-0.009576,-0.012493,-0.016903,-0.008848,-0.003355,-0.004436,-0.001393


In [9]:
y

,2
0,speech
1,speech
2,speech
3,song
4,speech
...,...
2447,song
2448,song
2449,song
2450,song


In [10]:
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

In [11]:
"""scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)
X = scaler.fit_transform(X).reshape(X.shape[0], X.shape[1])"""

'scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)\nX = scaler.fit_transform(X).reshape(X.shape[0], X.shape[1])'

In [12]:
X_train=X.iloc[0:1827,:]
X_test=X.iloc[1828:,:]
y_train=y.iloc[0:1827,:]
y_test=y.iloc[1828:,:]

In [13]:
X_train.shape

(1827, 3000)

In [14]:
len(X_test)

624

In [15]:
y_train

,2
0,speech
1,speech
2,speech
3,song
4,speech
...,...
1822,speech
1823,song
1824,speech
1825,speech


In [16]:
y_test

,2
1828,speech
1829,speech
1830,speech
1831,speech
1832,speech
...,...
2447,song
2448,song
2449,song
2450,song


In [17]:
"""X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=100, stratify=y)"""

'X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=100, stratify=y)'

In [18]:
y_train=y_train.T.iloc[0]
y_test=y_test.T.iloc[0]

In [19]:
len(y_train)

1827

In [20]:
y_train=y_train.copy()
for i in range(len(y_train)):##devo fare sto troiao perchè per plottare vuole in input un array con 0 o 1... capiamo poi se conviene fare questa operazione a livello di dataset
    if y_train[i]=='speech':
        y_train[i]=0
    else:
        y_train[i]=1
y_train=y_train.astype(int)

In [21]:
y_test=y_test.copy()
for i in range(1828,1828+len(y_test)):##devo fare sto troiao perchè per plottare vuole in input un array con 0 o 1... capiamo poi se conviene fare questa operazione a livello di dataset
    if y_test[i]=='speech':
        y_test[i]=0
    else:
        y_test[i]=1
y_test=y_test.astype(int)

In [22]:
y_test

1828    0
1829    0
1830    0
1831    0
1832    0
       ..
2447    1
2448    1
2449    1
2450    1
2451    1
Name: 2, Length: 624, dtype: int32

In [23]:
y_train

0       0
1       0
2       0
3       1
4       0
       ..
1822    0
1823    1
1824    0
1825    0
1826    0
Name: 2, Length: 1827, dtype: int32

In [24]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import EditedNearestNeighbours
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from sklearn.cluster import MiniBatchKMeans
from imblearn.under_sampling import ClusterCentroids

In [25]:
"""cc = ClusterCentroids(
    estimator=MiniBatchKMeans(n_init=1, random_state=0), random_state=42
)
X_res, y_res = cc.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))"""

"""sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train))"""

"""ada = ADASYN(random_state=42)
X_train, y_train = ada.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_train))"""

"ada = ADASYN(random_state=42)\nX_train, y_train = ada.fit_resample(X_train, y_train)\nprint('Resampled dataset shape %s' % Counter(y_train))"

In [26]:
len(X_train)

1827

In [27]:
"""len(X_train[0])"""

'len(X_train[0])'

In [28]:
y_train.shape

(1827,)

In [29]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
def rs_best_estimator(clf, param_grid, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = RandomizedSearchCV(estimator=clf, param_distributions=param_grid,n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
    grid_result = grid_search.fit(X, y)
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))    
    return grid_search

In [30]:
"""clf = KNeighborsClassifier()
p=[1,2]
n_neighbors = [1,2,3,4,5,6,7,8,9,10]
weights  = ['uniform', 'distance',]
param_grid1 = dict(n_neighbors=n_neighbors,weights=weights,p=p)
rs_results_logreg = rs_best_estimator(clf, param_grid1, X_train, y_train)"""

"clf = KNeighborsClassifier()\np=[1,2]\nn_neighbors = [1,2,3,4,5,6,7,8,9,10]\nweights  = ['uniform', 'distance',]\nparam_grid1 = dict(n_neighbors=n_neighbors,weights=weights,p=p)\nrs_results_logreg = rs_best_estimator(clf, param_grid1, X_train, y_train)"

In [31]:
"""clf = KNeighborsClassifier(p=2)
n_neighbors = [1,2,3]
weights  = ['uniform', 'distance',]
param_grid1 = dict(n_neighbors=n_neighbors,weights=weights)
rs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,375:750], y_train)"""

"clf = KNeighborsClassifier(p=2)\nn_neighbors = [1,2,3]\nweights  = ['uniform', 'distance',]\nparam_grid1 = dict(n_neighbors=n_neighbors,weights=weights)\nrs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,375:750], y_train)"

In [32]:
"""clf = KNeighborsClassifier(p=2)
n_neighbors = [1,2,3]
weights  = ['uniform', 'distance',]
param_grid1 = dict(n_neighbors=n_neighbors,weights=weights)
rs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,750:1125], y_train)"""

"clf = KNeighborsClassifier(p=2)\nn_neighbors = [1,2,3]\nweights  = ['uniform', 'distance',]\nparam_grid1 = dict(n_neighbors=n_neighbors,weights=weights)\nrs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,750:1125], y_train)"

In [33]:
"""clf = KNeighborsClassifier(p=2)
n_neighbors = [1,2,3]
weights  = ['uniform', 'distance',]
param_grid1 = dict(n_neighbors=n_neighbors,weights=weights)
rs_results_logreg = rs_best_estimator(clf, param_grid1, X_train, y_train)"""

"clf = KNeighborsClassifier(p=2)\nn_neighbors = [1,2,3]\nweights  = ['uniform', 'distance',]\nparam_grid1 = dict(n_neighbors=n_neighbors,weights=weights)\nrs_results_logreg = rs_best_estimator(clf, param_grid1, X_train, y_train)"

In [34]:
"""clf = KNeighborsClassifier(metric='dtw_sakoechiba')
n_neighbors = [1,2,3]
weights  = ['uniform', 'distance',]
param_grid1 = dict(n_neighbors=n_neighbors,weights=weights)
rs_results_logreg = rs_best_estimator(clf, param_grid1, X_train, y_train)"""

"clf = KNeighborsClassifier(metric='dtw_sakoechiba')\nn_neighbors = [1,2,3]\nweights  = ['uniform', 'distance',]\nparam_grid1 = dict(n_neighbors=n_neighbors,weights=weights)\nrs_results_logreg = rs_best_estimator(clf, param_grid1, X_train, y_train)"

In [35]:
"""clf = KNeighborsClassifier(metric='dtw_sakoechiba')
n_neighbors = [1,2,3]
weights  = ['uniform', 'distance',]
param_grid1 = dict(n_neighbors=n_neighbors,weights=weights)
rs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,375:750], y_train)"""

"clf = KNeighborsClassifier(metric='dtw_sakoechiba')\nn_neighbors = [1,2,3]\nweights  = ['uniform', 'distance',]\nparam_grid1 = dict(n_neighbors=n_neighbors,weights=weights)\nrs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,375:750], y_train)"

In [36]:
"""clf = KNeighborsClassifier(metric='dtw_sakoechiba')
n_neighbors = [1,2,3]
weights  = ['uniform', 'distance',]
param_grid1 = dict(n_neighbors=n_neighbors,weights=weights)
rs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,750:1125], y_train)"""

"clf = KNeighborsClassifier(metric='dtw_sakoechiba')\nn_neighbors = [1,2,3]\nweights  = ['uniform', 'distance',]\nparam_grid1 = dict(n_neighbors=n_neighbors,weights=weights)\nrs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,750:1125], y_train)"

In [37]:
"""clf = KNeighborsClassifier(metric='dtw_sakoechiba')
n_neighbors = [1,2,3]
weights  = ['uniform', 'distance',]
param_grid1 = dict(n_neighbors=n_neighbors,weights=weights)
rs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,1125:1500], y_train)"""

"clf = KNeighborsClassifier(metric='dtw_sakoechiba')\nn_neighbors = [1,2,3]\nweights  = ['uniform', 'distance',]\nparam_grid1 = dict(n_neighbors=n_neighbors,weights=weights)\nrs_results_logreg = rs_best_estimator(clf, param_grid1, X_train[:,1125:1500], y_train)"

In [38]:
clf = KNeighborsTimeSeriesClassifier(metric="euclidean", n_neighbors=3)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.7980769230769231
F1-score [0.80733945 0.78787879]
              precision    recall  f1-score   support

           0       0.90      0.73      0.81       360
           1       0.71      0.89      0.79       264

    accuracy                           0.80       624
   macro avg       0.80      0.81      0.80       624
weighted avg       0.82      0.80      0.80       624



In [39]:
metric=['dtw']
for metric in metric:
    clf = KNeighborsTimeSeriesClassifier(metric=metric, n_neighbors=5, n_jobs=-1)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)

    print("classification with " + metric)    
    print('Accuracy %s' % accuracy_score(y_test, y_pred))
    print('F1-score %s' % f1_score(y_test, y_pred, average=None))
    print(classification_report(y_test, y_pred))

classification with dtw
Accuracy 0.9631410256410257
F1-score [0.96737589 0.95764273]
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       360
           1       0.93      0.98      0.96       264

    accuracy                           0.96       624
   macro avg       0.96      0.97      0.96       624
weighted avg       0.96      0.96      0.96       624



In [ ]:
"""!pip install joblib"""

'!pip install joblib'

In [ ]:
"""import joblib"""

SyntaxError: unterminated string literal (detected at line 1) (540490653.py, line 1)

In [ ]:
"""joblib.dump()"""

##CNN

In [95]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, GlobalAveragePooling1D
from keras.layers import Conv1D, Activation, Conv1D, BatchNormalization

In [96]:
def build_simple_cnn(n_timesteps, n_outputs):
    model = Sequential()
    
    model.add(Conv1D(filters=16, kernel_size=8, activation='relu', input_shape=(n_timesteps, 1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(GlobalAveragePooling1D())
    
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


In [98]:
X_train_cnn = X_train
X_test_cnn = X_test

X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(X_train_cnn, y_train, test_size=0.2, stratify=y_train)

n_timesteps, n_outputs, n_features = X_train_cnn.shape[1], len(np.unique(y_train_cnn)), 1 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)

TIMESTEPS:  2998
N. LABELS:  2


In [99]:
cnn = build_simple_cnn(n_timesteps, n_outputs)

In [100]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2991, 16)          144       
                                                                 
 batch_normalization (BatchN  (None, 2991, 16)         64        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 2991, 16)          0         
                                                                 
 dropout (Dropout)           (None, 2991, 16)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2987, 32)          2592      
                                                                 
 batch_normalization_1 (Batc  (None, 2987, 32)         128       
 hNormalization)                                        

In [101]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

In [102]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
mc = ModelCheckpoint('best_model_cnn.h5', monitor='val_loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 16
mini_batch_size = int(min(X_train_cnn.shape[0]/10, batch_size))

In [103]:
history_cnn = cnn.fit(X_train_cnn, y_train_cnn, epochs=5, batch_size=mini_batch_size, callbacks=callbacks,
                      validation_data=(X_val_cnn, y_val_cnn)).history

Epoch 1/5
92/92 [==============================] - 34s 324ms/step - loss: 0.4461 - accuracy: 0.7775 - val_loss: 0.6250 - val_accuracy: 0.8251 - lr: 0.0010
Epoch 2/5
92/92 [==============================] - 30s 323ms/step - loss: 0.3286 - accuracy: 0.8508 - val_loss: 0.5216 - val_accuracy: 0.8798 - lr: 0.0010
Epoch 3/5
92/92 [==============================] - 30s 329ms/step - loss: 0.2641 - accuracy: 0.8960 - val_loss: 0.3941 - val_accuracy: 0.8907 - lr: 0.0010
Epoch 4/5
92/92 [==============================] - 30s 323ms/step - loss: 0.2376 - accuracy: 0.9028 - val_loss: 0.3203 - val_accuracy: 0.8579 - lr: 0.0010
Epoch 5/5
92/92 [==============================] - 30s 324ms/step - loss: 0.2150 - accuracy: 0.9199 - val_loss: 0.2263 - val_accuracy: 0.9372 - lr: 0.0010


In [104]:
y_pred = np.argmax(cnn.predict(X_test_cnn), axis=1)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

20/20 [==============================] - 2s 74ms/step
Accuracy 0.967948717948718
F1-score [0.9726776  0.96124031]
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       360
           1       0.98      0.94      0.96       264

    accuracy                           0.97       624
   macro avg       0.97      0.96      0.97       624
weighted avg       0.97      0.97      0.97       624



In [105]:
cnn.evaluate(X_test_cnn, y_test)

20/20 [==============================] - 2s 89ms/step - loss: 0.1728 - accuracy: 0.9679


[0.1728043556213379, 0.9679487347602844]

###SHAPELETS

In [106]:
from pyts.transformation import ShapeletTransform

In [107]:
# Shapelet transformation
st = ShapeletTransform(window_sizes=[4, 12],
                       random_state=42, sort=True)
X_new = st.fit_transform(X_train, y_train)
X_test_new = st.transform(X_test)

In [ ]:
"""from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=8, random_state=42)
clf.fit(X_new, y_train)

y_pred = clf.predict(X_test_new)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))"""